## Notes for Users

- **No Real-Time UI**: All monitoring is post-hoc. Run experiments first, then load logs here.
- **CSV/JSON Export**: Every episode is saved as structured data in `../logs/`.
- **Reproducibility**: Experiments are seeded; outputs should be deterministic.
- **Next Step**: Modify this notebook path or agent names to match your actual experiments.

In [1]:
if summary_file.exists():
    # Group by agent and compute statistics
    summary_df = pd.read_csv(summary_file)
    agent_stats = summary_df.groupby('agent')['total_revenue_euro'].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max'),
        ('episodes', 'count')
    ]).round(2)
    
    print("\nAgent Performance Comparison:")
    print(agent_stats)
    
    # Plot comparison
    if 'agent' in summary_df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        for agent in summary_df['agent'].unique():
            agent_data = summary_df[summary_df['agent'] == agent]['total_revenue_euro']
            ax.hist(agent_data, alpha=0.6, label=agent, bins=10)
        
        ax.set_xlabel('Total Revenue (€)', fontsize=10)
        ax.set_ylabel('Frequency', fontsize=10)
        ax.set_title('Revenue Distribution by Agent', fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.show()

NameError: name 'summary_file' is not defined

## 5. Multi-Agent Comparison (if available)

Compare multiple agents (e.g., RL vs. rule-based vs. MPC).

In [ ]:
# Load summary file if it exists
summary_file = REPORT_DIR / "summary.csv"
if summary_file.exists():
    summary_df = pd.read_csv(summary_file)
    print("Experiment Summary:")
    print(summary_df.to_string(index=False))
    print(f"\nSaved to: {summary_file}")
else:
    print("No summary.csv found. Run experiments to generate summaries.")

## 4. Aggregate Results Across Episodes

Compare total revenue and action distributions across all episodes.

In [ ]:
if csv_logs:
    fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)
    
    # Plot 1: Electricity Price
    axes[0].plot(episode_df['timestep'], episode_df['price_euro_per_mwh'], 'b-', linewidth=1.5)
    axes[0].set_ylabel('Price (€/MWh)', fontsize=10)
    axes[0].set_title(f'Episode Trajectory: {episode_csv.name}', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: State of Charge with action coloring
    charge_mask = episode_df['action_label'] == 'charge'
    discharge_mask = episode_df['action_label'] == 'discharge'
    idle_mask = episode_df['action_label'] == 'idle'
    
    axes[1].plot(episode_df['timestep'], episode_df['soc_percent'], 'k-', linewidth=2, label='SoC')
    axes[1].scatter(episode_df[charge_mask]['timestep'], episode_df[charge_mask]['soc_percent'], 
                   c='green', s=20, alpha=0.6, label='Charging')
    axes[1].scatter(episode_df[discharge_mask]['timestep'], episode_df[discharge_mask]['soc_percent'], 
                   c='red', s=20, alpha=0.6, label='Discharging')
    axes[1].scatter(episode_df[idle_mask]['timestep'], episode_df[idle_mask]['soc_percent'], 
                   c='gray', s=10, alpha=0.4, label='Idle')
    axes[1].set_ylabel('State of Charge (%)', fontsize=10)
    axes[1].set_ylim([0, 105])
    axes[1].legend(loc='best', fontsize=8)
    axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Cumulative Revenue
    axes[2].plot(episode_df['timestep'], episode_df['cumulative_revenue_euro'], 'g-', linewidth=2)
    axes[2].fill_between(episode_df['timestep'], episode_df['cumulative_revenue_euro'], alpha=0.3, color='green')
    axes[2].set_ylabel('Cumulative Revenue (€)', fontsize=10)
    axes[2].set_xlabel('Time Step (hours)', fontsize=10)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 3. Plot Episode Trajectory

Visualize price signal, SoC, and cumulative revenue over time.

In [ ]:
# Load first episode (edit filename if needed)
if csv_logs:
    episode_csv = csv_logs[0]
    episode_df = pd.read_csv(episode_csv)
    
    print(f"\nEpisode: {episode_csv.name}")
    print(f"Duration: {len(episode_df)} steps")
    print(f"\nFirst 5 rows:")
    print(episode_df.head())
    print(f"\nLast 5 rows:")
    print(episode_df.tail())
    
    print(f"\nSummary Statistics:")
    print(f"  Electricity Price: €{episode_df['price_euro_per_mwh'].min():.2f} to €{episode_df['price_euro_per_mwh'].max():.2f}")
    print(f"  SoC Range: {episode_df['soc_percent'].min():.1f}% to {episode_df['soc_percent'].max():.1f}%")
    print(f"  Final Cumulative Revenue: €{episode_df['cumulative_revenue_euro'].iloc[-1]:.2f}")
    print(f"  Action Distribution:")
    print(f"    {episode_df['action_label'].value_counts().to_dict()}")
else:
    print("No episode logs found. Run experiments first.")

## 2. Inspect One Episode

Load and display a single episode trajectory.

In [ ]:
# List available episode logs
csv_logs = sorted(LOG_DIR.glob("*.csv")) if LOG_DIR.exists() else []
print(f"Found {len(csv_logs)} episode logs:")
for log in csv_logs[:5]:  # Show first 5
    print(f"  {log.name}")
if len(csv_logs) > 5:
    print(f"  ... and {len(csv_logs) - 5} more")

## 1. Load Episode Logs

CSV files contain step-by-step records: price, SoC, action, reward, cumulative revenue.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Set paths
LOG_DIR = Path("../logs")
REPORT_DIR = Path("../reports")

print(f"Log directory: {LOG_DIR}")
print(f"Report directory: {REPORT_DIR}")

# BESS Experiment Analysis
## Post-hoc Inspection of Episode Trajectories

This notebook loads and visualizes experiment logs from CSV/JSON files.
No live dashboard—pure research-oriented data inspection.